In [ ]:
import matplotlib.pyplot as plt
import readline
import GPy
from SAM.SAM_Core import SAMCore
from SAM.SAM_Core import SAMDriver
import pylab as pb
import sys 
from sys import executable
import subprocess
from subprocess import Popen, PIPE
import pickle
import os
from os import listdir, walk, system
from os.path import isfile, join, isdir
import time
import operator
import numpy as np
import numpy.ma as ma
import datetime
import yarp
import copy
from itertools import combinations 
from ConfigParser import SafeConfigParser
from scipy.spatial import distance
from numpy.linalg import inv
import math

dataSetFolder = '/home/daniel/WYSIWYD_PROJECT/actionRecognitionDataset'
dataFolderList = ['lift-drop-right_arm', 
                  'lift-drop-left_arm', 
                  'push-pull-left_arm', 
                  'push-pull-right_arm']

dataFolders = []
labelFolders = []
#check folders in list actually exist
for j in dataFolderList:
    t = join(dataSetFolder,j,'data')
    m = join(dataSetFolder,j,'labels')
    if(isdir(t) and isdir(m)):
        dataFolders.append(t)
        labelFolders.append(m)
print '\n'.join(dataFolders)
print
print '\n'.join(labelFolders)
#subDirFolderList = ['camera/left', 'camera/right', 'data', 'kinect/rgb', 'labels']


In [ ]:
#load kinect skeleton data for all folders in dataFolderList
numJoints = 9
data = dict()
firstPass = True
jointsList = []
objectsList = []
labelsList = []
numFolders = len(dataFolders)

for j in dataFolders:
    k = dataFolders.index(j)
    dataFile = open(join(dataFolders[k],'data.log'),'r')
    labelFile = open(join(labelFolders[k],'data.log'),'r')
    
    #number of lines in dataFile
    for i, l in enumerate(dataFile):
            pass
    lenDataFile = i+1
    
    #number of lines in labelFile
    for i, l in enumerate(labelFile):
            pass
    lenLabelFile = i+1
    dataFile.close()
    labelFile.close()
    
    
    if(lenLabelFile != lenDataFile):
        raise ValueError('Files not of same lenght')
    else:
        dataFile = open(join(dataFolders[k],'data.log'),'r')
        labelFile = open(join(labelFolders[k],'data.log'),'r')
        labelsList.append([])
        
        for curr in range(lenDataFile):
            line = dataFile.readline()
            labelLine = labelFile.readline()
            
            t = line.replace('(','').replace(')','').split(' ')
            del t[0:4]
            #parse skeleton data which has 9 sections by (x,y,z)
            for i in range(numJoints):
                a = i*4
                if(firstPass):
                    data[t[a]] = [None]*numFolders
                    data[t[a]][k] = (np.array([float(t[a+1]), float(t[a+2]), float(t[a+3])]))
                    jointsList.append(t[a])
                else:
                    arr =  np.array([float(t[a+1]), float(t[a+2]), float(t[a+3])])
                    if(data[t[a]][k] != None):
                        data[t[a]][k] = np.vstack((data[t[a]][k],arr))
                    else:
                        data[t[a]][k] = arr

            currIdx = (numJoints*4 -1)
            numObjs = (len(t) - currIdx)/5

            for i in range(numObjs):
                a = currIdx + 1 + (i*5)
                if(t[a] in data):
                    arr = np.array([float(t[a+1]), float(t[a+2]), float(t[a+3])])
                    if(data[t[a]][k] != None):
                        data[t[a]][k] =  np.vstack((data[t[a]][k],arr))
                    else:
                        data[t[a]][k] = arr
                else:
                    data[t[a]] = [None]*(numFolders+1)
                    data[t[a]][k] = np.array([float(t[a+1]), float(t[a+2]), float(t[a+3])])
                    data[t[a]][-1] = int(t[a+4])
                    objectsList.append(t[a])

            firstPass = False
            try:
                v = labelLine.split(' ')[2].replace('\n','').replace('(','').replace(')','')
            except IndexError:
                print labelLine
                
            labelsList[k].append(v)
            
        dataFile.close()
        labelFile.close()

#compile a list of all unique labels
setList = []
for x in labelsList:
    setList.append(list(set(x)))
flattenedList = [val for sublist in setList for val in sublist]
labels = list(set(flattenedList))
labels.sort()
for k in range(0,len(labels)):
    print str(k) + '  ' + labels[k]

In [ ]:
#prepare list of indices for labels which will be used
ignoreInds = [1,2,9] #agent_entry, agent_exit, no_agent
#important that no other labels are removed because that would interfere with temporal continuity of data
#no temporal continuity would make QTC calculation with big jumps which is not desirable
#future work needs to go through data and split it into subsections depending on actions to be considered

doLabels = [x for i,x in enumerate(labels) if i not in ignoreInds]
indicesList = []
for ll in labelsList:
    indicesList.append([i for i, x in enumerate(ll) if x in doLabels])

In [ ]:
#apply indices
subsetData = None
subsetLabels = None
subsetData = copy.deepcopy(data)
subsetLabels = copy.deepcopy(labelsList)

for k in range(numFolders):
    for j in jointsList + objectsList:
        subsetData[j][k] = np.squeeze(data[j][k][[indicesList[k]],:])
        subsetLabels[k] = [labelsList[k][i] for i in indicesList[k]]
# data = None
# labelsList = None

In [ ]:
#apply indices
count = 0
off1 = 3
off2 = 2
off3 = 15
off4 = 25
for k in range(numFolders):
    for j in jointsList + objectsList:
        count += 1
        print str(count).ljust(off1) + ' Folder ' + str(k).ljust(off2) + ' object: ' + j.ljust(off3) + \
        ' data shape: '.ljust(off4) + str(len(data)) + ' ' + str(len(data[j])) + ' ' + str(data[j][k].shape)
        print str(count).ljust(off1) + ' Folder ' + str(k).ljust(off2) + ' object: ' + j.ljust(off3) + \
        ' subset data shape: '.ljust(off4) + str(len(subsetData)) + ' ' + str(len(subsetData[j])) + ' ' + str(subsetData[j][k].shape)
        print str(count).ljust(off1) + ' Folder ' + str(k).ljust(off2) + ' object: ' + j.ljust(off3) + \
        ' labels shape: '.ljust(off4) + str(len(labelsList)) + '     ' + str(len(labelsList[k]))
        print str(count).ljust(off1) + ' Folder ' + str(k).ljust(off2) + ' object: ' + j.ljust(off3) + \
        ' subset labels shape: '.ljust(off4) + str(len(subsetLabels)) + '     ' + str(len(subsetLabels[k]))
        print str(count).ljust(off1) + ' Folder ' + str(k).ljust(off2) + ' object: ' + j.ljust(off3) + \
        ' good inds shape: '.ljust(off4) + str(len(indicesList)) + '     ' + str(len(indicesList[k]))
        print
        


In [ ]:
#so far the data has been first formatted into a dictionary for the joints and objects with each key holding
#numFolders numpyarrays, one for each data folder. labels are formatted into a list with numFolders lenght
#no_agent, agent_entry and agent_exit have been removed and the remaining data stored in subsetData, subsetLabels
#next step create combinatorial pairs

In [ ]:
allObjs = jointsList + objectsList
print '\n'.join(allObjs)

In [ ]:
#select which combinations are importantd
#shoulderCentre-handLeft shoulderCentre-handRight handLeft-activeObject handRight-activeObject

#remove items wich are not important
impObjs = copy.deepcopy(allObjs)

remObjs = ['partner','car','octopus']

for x in remObjs:
    impObjs.remove(x)
print '\n'.join(impObjs)
print 
objCombs = list(combinations(impObjs, 2))
print objCombs
print len(objCombs)

In [ ]:
def distEuc(a,b):
    temp = a-b
    temp = np.square(temp)
    temp = np.sum(temp,1)
    return np.sqrt(temp)

def qtc_2D(k,l,q,thresh, contactThresh, contact = None):
    
    d1 = distEuc(k[:-2],l[1:-1])
    d2 = distEuc(k[1:-1],l[1:-1])
    d3 = distEuc(k[2:],l[1:-1])
    
    func2 = 0
    if(func2 == 0):
        for i in range(len(d1)):
            #threshold distance moved
            diff1 = d2[i]-d1[i]
            if(np.abs(diff1) < thresh):
                diff1 = 0

            diff2 = d3[i]-d2[i]
            if(np.abs(diff2) < thresh):
                diff2 = 0

            #convert to qtc
            if(diff1 > 0 and diff2 > 0):
                q[i] = -1
            elif(diff1 < 0 and diff2 < 0):
                q[i] = +1
            else:
                q[i] = 0
                
    elif(func2 == 1):
            #threshold distances
        inds = d1 < thresh
        d1[inds] = 0

        inds = d2 < thresh
        d2[inds] = 0

        inds = d3 < thresh
        d3[inds] = 0

        for i in range(len(d1)):
            if(d1[i] < d2[i] < d3[i]):
                q[i] = -1
            elif(d1[i] > d2[i] > d3[i]):
                q[i] = +1
            else:
                q[i] = 0
        
        #check contact
        
        #check qtc smoothness

def frenetFrame(arr):
    t_num = np.diff(arr,axis=0)
    t = (t_num/np.abs(t_num)).astype(int)

    b_num = np.cross(t[:-1],t[1:])
    b = b_num/np.abs(b_num)
    t = t[1:]

    n = np.cross(b,t)

    frameArr = np.concatenate((t,n,b),axis=1).T
    fArr = frameArr.reshape((3,3,-1),order = 'F')
    return fArr

def qtc_3D(k, l, thresh, q3, q4, q5):
    fFrameK = frenetFrame(k)
    fFrameL = frenetFrame(l)

    for g in range(fFrameK.shape[2]):
        fKinv = np.linalg.pinv(fFrameK[:,:,g])
        R = np.dot(fFrameL[:,:,g],fKinv)
        
        alpha = np.arctan(R[1,0]/R[0,0])
        den = np.sqrt(pow(R[2,1],2) + pow(R[2,2],2))
        
        beta = np.arctan(-R[2,0]/den)
        gamma = np.arctan(R[2,1]/R[2,2])

        #threshold angles
        if(np.abs(alpha) < thresh or math.isnan(alpha)):
            alpha = 0
        if(np.abs(beta) < thresh or math.isnan(beta)):
            beta = 0
        if(np.abs(gamma) < thresh or math.isnan(gamma)):
            gamma = 0

        q3[g] = np.sign(alpha)
        q4[g] = np.sign(beta)
        q5[g] = np.sign(gamma)

In [ ]:
qtcDataList = dict()
angleThreshold = 0.1
distanceThreshold = 0.001
contactThreshold = 0.01
allJoints = []
#for all combs
for arr in range(len(subsetData[objCombs[0][0]])):
    jointArr = None
    for currComb in objCombs:
        Pk = subsetData[currComb[0]]
        Pl = subsetData[currComb[1]]
        #for all arrays in Pk and Pl
        currPk = Pk[arr]
        currPl = Pl[arr]

        q1 = np.zeros(currPk.shape[0]-2, dtype=np.int)
        q2 = np.zeros(currPk.shape[0]-2, dtype=np.int)
        q3 = np.zeros(currPk.shape[0]-2, dtype=np.int)
        q4 = np.zeros(currPk.shape[0]-2, dtype=np.int)
        q5 = np.zeros(currPk.shape[0]-2, dtype=np.int)
        #currPk contains xyz and currPl contains xyz
        #calculate QTC
        #step 1: q1 = {-1,0,+1} Pk relative to Pl
        qtc_2D(currPk,currPl,q1, distanceThreshold, contactThreshold)
        #step 2: q2 = {-1,0,+1} Pl relative to Pk
        qtc_2D(currPl,currPk,q2, distanceThreshold, contactThreshold)
        #step 3: calculate q3, q4 and q5
        qtc_3D(currPk, currPl, angleThreshold, q3, q4, q5)

        tempArr = np.vstack((q1,q2,q3,q4,q5))
        if(jointArr == None):
            jointArr = tempArr.T
        else:
            jointArr = np.hstack((jointArr,tempArr.T))
    allJoints.append(jointArr)
    
    #updating labels
    subsetLabels[arr] = subsetLabels[arr][1:-1]



In [ ]:
mask = np.zeros(allJoints[0].shape[1], dtype = int)
for i in range(0,len(mask),5):
    mask[i+0] = 1
    mask[i+1] = 1

In [ ]:
actionsIdxList = []
actionsLabelsList = []

#for each item in list
for k in range(len(allJoints)):
    actionsIdxList.append([])
    actionsLabelsList.append([])
    #create array that masks q2,q3 and q4
    maJoints = allJoints[k]*mask
    #sum abs(rows) of array
    maAbs = np.abs(maJoints)
    maSum = np.sum(maAbs, axis = 1)
    #indices with maAbs =  0 are regions with no movement 
    #.ie action demarcation between actions
    #maSum = maSum[:50]
    actionCount = 0
    actionsIndices = None
    actionsLabels = []
    started = False
    for n in range(len(maSum)):
        if(maSum[n] == 0):
            #here we need to close action if previous is not zero
            #start action if next is not zero
            #ignore otherwise

            #if n-1 not 0 end action
            #check n-1 exists 
            if(n-1 >= 0):
                if(maSum[n-1] != 0):
                    #end action including 0 at n
                    actionsIndices = np.hstack((actionsIndices, allJoints[k][n]))
                    actionsLabels.append(subsetLabels[k][n])
                    started = False
                    actionsIdxList[k].append(actionsIndices)
                    actionsLabelsList[k].append(actionsLabels)
                    actionsIndices = None
                    actionsLabels = []

            #check n+1 exists
            if(n+1 < len(maSum)):
                #if n+1 exists but not 0 start an action
                if(maSum[n+1] != 0):
                    #start action including 0 at n
                    actionsIndices = allJoints[k][n]
                    actionsLabels.append(subsetLabels[k][n])
                    started = True
                #else ignore current index
            #else we are at the end so ignore

        else: #current index is not zero
            if(started):
                #here if started = True we are in middle of action so concatenate
                actionsIndices = np.hstack((actionsIndices, allJoints[k][n]))
                actionsLabels.append(subsetLabels[k][n])
                if(n+1 == len(maSum)):
                    actionsIdxList[k].append(actionsIndices)
                    actionsLabelsList[k].append(actionsLabels)
            else:
                #here action has not started meaning a zero was not found
                #this occurs if vector does not start with a zero
                actionsIndices = allJoints[k][n]
                actionsLabels.append(subsetLabels[k][n])
                started = True

In [ ]:
for h in range(len(actionsIdxList)):
    for p in actionsLabelsList[h]:
        print p
        print
        print
        print

In [ ]:
#find maximum length vector for SAM
maxLen = 0
for n in actionsIdxList:
    for k in n:
        if(k.shape[0] > maxLen):
            maxLen = k.shape[0]

# print maxLen

#create Y and L
Y = None
L = []
for n in range(len(actionsIdxList)):
    for k in range(len(actionsIdxList[n])):
        currLen = len(actionsIdxList[n][k])
        augMat = np.zeros(maxLen-currLen)
        if(Y == None): 
            Y = np.hstack((actionsIdxList[n][k],augMat))
        else:
            Y = np.vstack((Y, np.hstack((actionsIdxList[n][k],augMat))))
        L.append(actionsLabelsList[n][k])
print Y.shape
print len(L)

In [ ]:
import itertools
import operator

def most_common(L):
    # get an iterable of (item, iterable) pairs
    SL = sorted((x, i) for i, x in enumerate(L))
    # print 'SL:', SL
    groups = itertools.groupby(SL, key=operator.itemgetter(0))
    # auxiliary function to get "quality" for an item
    def _auxfun(g):
        item, iterable = g
        count = 0
        min_index = len(L)
        for _, where in iterable:
            count += 1
            min_index = min(min_index, where)
            # print 'item %r, count %r, minind %r' % (item, count, min_index)
        return count, -min_index
    # pick the highest-count/earliest item
    return max(groups, key=_auxfun)[0]

In [ ]:
print 

In [ ]:
L2 = [most_common(sublist) for sublist in L]

Larr = np.zeros(len(L2))
for f in range(len(L2)):
    Larr[f] = labels.index(L2[f])

print labels
print Larr.shape

In [ ]:
class AR_Driver(SAMDriver):
    def testing(self, testInstance):
        # Returns the predictive mean, the predictive variance and the axis (pp) of the latent space backwards mapping.            
        #mm,vv,pp=self.SAMObject.pattern_completion(testFace, visualiseInfo=visualiseInfo)
        ret=self.SAMObject.pattern_completion(testInstance, visualiseInfo=None)

        mm = ret[0]
        vv = ret[1]
        post = ret[3]        

        # find nearest neighbour of mm and SAMObject.model.X
        dists = np.zeros((self.SAMObject.model.X.shape[0],1))

        for j in range(dists.shape[0]):
            dists[j,:] = distance.euclidean(self.SAMObject.model.X.mean[j,:], mm[0].values)
        nn, min_value = min(enumerate(dists), key=operator.itemgetter(1))
        if self.SAMObject.type == 'mrd':
            print "With " + str(vv.mean()) +" prob. error the new action is " + labels[int(self.SAMObject.model.bgplvms[1].Y[nn,:])]
            textStringOut=labels[int(self.SAMObject.model.bgplvms[1].Y[nn,:])]

        elif self.SAMObject.type == 'bgplvm':
            print "With " + str(vv.mean()) +" prob. error the new action is " + labels[int(self.L[nn,:])]
            textStringOut=labels[int(self.L[nn,:])]

        if(vv.mean()<0.00012):            
            print "The action is " + textStringOut
        elif(vv.mean()>0.00012):
            print "I think the action is " + textStringOut + " but I am not sure"      

        # # Plot the training NN of the test image (the NN is found in the INTERNAl, compressed (latent) memory space!!!)
        # if visualiseInfo is not None:
        #     fig_nn = visualiseInfo['fig_nn']
        #     fig_nn = pb.figure(11)
        #     pb.title('Training NN')
        #     fig_nn.clf()
        #     pl_nn = fig_nn.add_subplot(111)
        #     pl_nn.imshow(numpy.reshape(self.SAMObject.recall(nn),(self.imgHeightNew, self.imgWidthNew)), cmap=plt.cm.Greys_r)
        #     pb.title('Training NN')
        #     pb.show()
        #     pb.draw()
        #     pb.waitforbuttonpress(0.1)
        #return pp

        return ret[2]
    
    def readData(self, root_data_dir, participant_index, Y, L):
        self.Y = Y
        self.L = L[:,None]
    
    

In [ ]:
mySAMpy = AR_Driver()
mySAMpy.readData(0,0,Y,Larr)

Ntr = int(len(mySAMpy.L)*40/100)
model_type = 'mrd'
model_num_inducing = 35
model_num_iterations = 100 #100
model_init_iterations = 300 #800
save_model=True
economy_save = True
visualise_output=True
experiment_number = 1
fname = '/home/daniel/WYSIWYD_PROJECT/actionRecognitionDataset/model/mAR_exp' + str(experiment_number)

[Yall,Lall,YtestAll,LtestAll] = mySAMpy.prepareData(model_type, Ntr,randSeed=experiment_number)


In [ ]:
mySAMpy.training(model_num_inducing, model_num_iterations, model_init_iterations, fname, save_model, economy_save)


In [ ]:
ss = [];
sstest = [];
print

print 'Yall = ' + str(Yall.shape)
print 'Lall = ' + str(Lall.shape)
print 'YtestAll = ' + str(YtestAll.shape)
print 'LtestAll = ' + str(LtestAll.shape)

currTestData = YtestAll[0][None,:]
currLabel = LtestAll[0]

print 'curData = ' + str(currTestData.shape) 
print 'curLabel = ' + str(currLabel)
print 'curAction truth = ' + labels[int(currLabel)]


pp = mySAMpy.testing(currTestData)
print pp

In [ ]:
result = True

In [ ]:
print 'result' + str(result)

In [ ]:

t = yarp.Network.init()

In [ ]:
print t